Proving conjecture 2 with example

In [10]:
import numpy as np
from pulp import LpProblem, LpMinimize, LpVariable, LpStatus, value

In [12]:
# Function to solve the Lp problem for DSOP
def dsop_test(A_12, A_13, A_21, A_23, A_31, A_32):
    
    # Extract values from original payoff matrices
    A_011_21 = A_21[0, 0]
    A_012_21 = A_21[0, 1]
    A_021_21 = A_21[1, 0]
    A_022_21 = A_21[1, 1]
        
    A_011_31 = A_31[0, 0]
    A_012_31 = A_31[0, 1]
    A_021_31 = A_31[1, 0]
    A_022_31 = A_31[1, 1]
    
    A_11_12 = A_12[0, 0]
    A_12_12 = A_12[0, 1]
    A_21_12 = A_12[1, 0]
    A_22_12 = A_12[1, 1]
        
    A_11_13 = A_13[0, 0]
    A_12_13 = A_13[0, 1]
    A_21_13 = A_13[1, 0]
    A_22_13 = A_13[1, 1]
    
    A_11_32 = A_32[0, 0]
    A_12_32 = A_32[0, 1]
    A_21_32 = A_32[1, 0]
    A_22_32 = A_32[1, 1]
        
    A_11_23 = A_23[0, 0]
    A_12_23 = A_23[0, 1]
    A_21_23 = A_23[1, 0]
    A_22_23 = A_23[1, 1]

    # Define the LP problem
    dsop = LpProblem("Minimize_cost", LpMinimize)
    
    # Define the variables
    # Representing infinity norms
    alpha2 = LpVariable('alpha2', lowBound=0)
    alpha3 = LpVariable('alpha3', lowBound=0)
    # Representing magnitudes of the entries of the difference between the manipulated matrices and original matrices, which includes C3
    alpha_2_11 = LpVariable('alpha_2_11', lowBound=0)
    alpha_2_12 = LpVariable('alpha_2_12', lowBound=0)
    alpha_2_21 = LpVariable('alpha_2_21', lowBound=0)
    alpha_2_22 = LpVariable('alpha_2_22', lowBound=0)   
    alpha_3_11 = LpVariable('alpha_3_11', lowBound=0)
    alpha_3_12 = LpVariable('alpha_3_12', lowBound=0)
    alpha_3_21 = LpVariable('alpha_3_21', lowBound=0)
    alpha_3_22 = LpVariable('alpha_3_22', lowBound=0)
    # Entries of the manipulated matrices
    A_11_21 = LpVariable('A_11_21')
    A_12_21 = LpVariable('A_12_21')
    A_21_21 = LpVariable('A_21_21')
    A_22_21 = LpVariable('A_22_21')
    A_11_31 = LpVariable('A_11_31')
    A_12_31 = LpVariable('A_12_31')
    A_21_31 = LpVariable('A_21_31')
    A_22_31 = LpVariable('A_22_31')
    
    # Define the objective function
    dsop += alpha2 + alpha3
    
    # Define the constraints
    # C1
    dsop += A_11_21 - A_011_21 >= alpha_2_11
    dsop += A_12_21 - A_012_21 >= alpha_2_12
    dsop += A_21_21 - A_021_21 >= alpha_2_21
    dsop += A_22_21 - A_022_21 >= alpha_2_22
    dsop += A_11_31 - A_011_31 >= alpha_3_11
    dsop += A_12_31 - A_012_31 >= alpha_3_12
    dsop += A_21_31 - A_021_31 >= alpha_3_21
    dsop += A_22_31 - A_022_31 >= alpha_3_22
    # C2
    dsop += A_011_21 - A_11_21 >= alpha_2_11
    dsop += A_012_21 - A_12_21 >= alpha_2_12
    dsop += A_021_21 - A_21_21 >= alpha_2_21
    dsop += A_022_21 - A_22_21 >= alpha_2_22
    dsop += A_011_31 - A_11_31 >= alpha_3_11
    dsop += A_012_31 - A_12_31 >= alpha_3_12
    dsop += A_021_31 - A_21_31 >= alpha_3_21
    dsop += A_022_31 - A_22_31 >= alpha_3_22
    # C4
    dsop += alpha_2_11 + alpha_2_12 <= alpha2
    dsop += alpha_2_21 + alpha_2_22 <= alpha2
    dsop += alpha_3_11 + alpha_3_12 <= alpha3
    dsop += alpha_3_21 + alpha_3_22 <= alpha3
    # C5
    dsop += A_12_21 + A_22_23 <= A_12_12 + A_12_13 
    dsop += A_12_31 + A_22_32 <= A_12_12 + A_12_13 
    # C6
    epsilon = 10 ** (-6)
    dsop += A_11_21 + A_11_23 + epsilon <= A_12_21 + A_21_23
    dsop += A_11_21 + A_12_23 + epsilon <= A_12_21 + A_22_23
    dsop += A_11_31 + A_11_32 + epsilon <= A_12_31 + A_12_32
    dsop += A_11_31 + A_21_32 + epsilon <= A_12_31 + A_22_32
        
    # Solve the problem
    dsop.solve()

    # Give different returns based on feasibility
    if LpStatus[dsop.status] == 'Optimal':
        A_21 = np.array([value(A_11_21), value(A_12_21), value(A_21_21), value(A_22_21)])
        A_31 = np.array([value(A_11_31), value(A_12_31), value(A_21_31), value(A_22_31)])
        return value(dsop.objective), ('e_1', A_21, A_31)
    else:
        return LpStatus[dsop.status]

In [14]:
# Function to solve the Lp problem for DSBCOP
def dsbcop_test(A_12, A_13, A_21, A_23, A_31, A_32):
    
    # Extract values from matrices A_21 and A_31
    A_011_21 = A_21[0, 0]
    A_012_21 = A_21[0, 1]
    A_021_21 = A_21[1, 0]
    A_022_21 = A_21[1, 1]
        
    A_011_31 = A_31[0, 0]
    A_012_31 = A_31[0, 1]
    A_021_31 = A_31[1, 0]
    A_022_31 = A_31[1, 1]
    
    A_11_12 = A_12[0, 0]
    A_12_12 = A_12[0, 1]
    A_21_12 = A_12[1, 0]
    A_22_12 = A_12[1, 1]
        
    A_11_13 = A_13[0, 0]
    A_12_13 = A_13[0, 1]
    A_21_13 = A_13[1, 0]
    A_22_13 = A_13[1, 1]
    
    A_11_32 = A_32[0, 0]
    A_12_32 = A_32[0, 1]
    A_21_32 = A_32[1, 0]
    A_22_32 = A_32[1, 1]
        
    A_11_23 = A_23[0, 0]
    A_12_23 = A_23[0, 1]
    A_21_23 = A_23[1, 0]
    A_22_23 = A_23[1, 1]

    # Define the LP problem
    dsbcop = LpProblem("Minimize_cost", LpMinimize)
    
    # Define the variables
    # Representing infinity norms
    alpha2_2 = LpVariable('alpha2_2', lowBound=0)
    alpha3_2 = LpVariable('alpha3_2', lowBound=0)
    alpha2_3 = LpVariable('alpha2_3', lowBound=0)
    alpha3_3 = LpVariable('alpha3_3', lowBound=0)
    # Representing magnitudes of the entries of the difference between the manipulated matrices and original matrices, which includes C9
    alpha_2_11_2 = LpVariable('alpha_2_11_2', lowBound=0)
    alpha_2_12_2 = LpVariable('alpha_2_12_2', lowBound=0)
    alpha_2_21_2 = LpVariable('alpha_2_21_2', lowBound=0)
    alpha_2_22_2 = LpVariable('alpha_2_22_2', lowBound=0)
    alpha_2_11_3 = LpVariable('alpha_2_11_3', lowBound=0)
    alpha_2_12_3 = LpVariable('alpha_2_12_3', lowBound=0)
    alpha_2_21_3 = LpVariable('alpha_2_21_3', lowBound=0)
    alpha_2_22_3 = LpVariable('alpha_2_22_3', lowBound=0)
    alpha_3_11_2 = LpVariable('alpha_3_11_2', lowBound=0)
    alpha_3_12_2 = LpVariable('alpha_3_12_2', lowBound=0)
    alpha_3_21_2 = LpVariable('alpha_3_21_2', lowBound=0)
    alpha_3_22_2 = LpVariable('alpha_3_22_2', lowBound=0)
    alpha_3_11_3 = LpVariable('alpha_3_11_3', lowBound=0)
    alpha_3_12_3 = LpVariable('alpha_3_12_3', lowBound=0)
    alpha_3_21_3 = LpVariable('alpha_3_21_3', lowBound=0)
    alpha_3_22_3 = LpVariable('alpha_3_22_3', lowBound=0)
    # Entries of the manipulated matrices
    A_211_21 = LpVariable('A_211_21')
    A_212_21 = LpVariable('A_212_21')
    A_221_21 = LpVariable('A_221_21')
    A_222_21 = LpVariable('A_222_21')
    A_311_21 = LpVariable('A_311_21')
    A_312_21 = LpVariable('A_312_21')
    A_321_21 = LpVariable('A_321_21')
    A_322_21 = LpVariable('A_322_21')
    A_211_31 = LpVariable('A_211_31')
    A_212_31 = LpVariable('A_212_31')
    A_221_31 = LpVariable('A_221_31')
    A_222_31 = LpVariable('A_222_31')
    A_311_31 = LpVariable('A_311_31')
    A_312_31 = LpVariable('A_312_31')
    A_321_31 = LpVariable('A_321_31')
    A_322_31 = LpVariable('A_322_31')
    
    # Define the objective function
    dsbcop += alpha2_2 + alpha3_2 + alpha2_3 + alpha3_3 
    
    # Define the constraints
    # C7
    dsbcop += A_211_21 - A_011_21 >= alpha_2_11_2
    dsbcop += A_212_21 - A_012_21 >= alpha_2_12_2
    dsbcop += A_221_21 - A_021_21 >= alpha_2_21_2
    dsbcop += A_222_21 - A_022_21 >= alpha_2_22_2
    dsbcop += A_211_31 - A_011_31 >= alpha_3_11_2
    dsbcop += A_212_31 - A_012_31 >= alpha_3_12_2
    dsbcop += A_221_31 - A_021_31 >= alpha_3_21_2
    dsbcop += A_222_31 - A_022_31 >= alpha_3_22_2
    dsbcop += A_311_21 - A_011_21 >= alpha_2_11_3
    dsbcop += A_312_21 - A_012_21 >= alpha_2_12_3
    dsbcop += A_321_21 - A_021_21 >= alpha_2_21_3
    dsbcop += A_322_21 - A_022_21 >= alpha_2_22_3
    dsbcop += A_311_31 - A_011_31 >= alpha_3_11_3
    dsbcop += A_312_31 - A_012_31 >= alpha_3_12_3
    dsbcop += A_321_31 - A_021_31 >= alpha_3_21_3
    dsbcop += A_322_31 - A_022_31 >= alpha_3_22_3
    # C8
    dsbcop += A_011_21 - A_211_21 >= alpha_2_11_2
    dsbcop += A_012_21 - A_212_21 >= alpha_2_12_2
    dsbcop += A_021_21 - A_221_21 >= alpha_2_21_2
    dsbcop += A_022_21 - A_222_21 >= alpha_2_22_2
    dsbcop += A_011_31 - A_211_31 >= alpha_3_11_2
    dsbcop += A_012_31 - A_212_31 >= alpha_3_12_2
    dsbcop += A_021_31 - A_221_31 >= alpha_3_21_2
    dsbcop += A_022_31 - A_222_31 >= alpha_3_22_2
    dsbcop += A_011_21 - A_311_21 >= alpha_2_11_3
    dsbcop += A_012_21 - A_312_21 >= alpha_2_12_3
    dsbcop += A_021_21 - A_321_21 >= alpha_2_21_3
    dsbcop += A_022_21 - A_322_21 >= alpha_2_22_3
    dsbcop += A_011_31 - A_311_31 >= alpha_3_11_3
    dsbcop += A_012_31 - A_312_31 >= alpha_3_12_3
    dsbcop += A_021_31 - A_321_31 >= alpha_3_21_3
    dsbcop += A_022_31 - A_322_31 >= alpha_3_22_3
    # C10
    dsbcop += alpha_2_11_2 + alpha_2_12_2 <= alpha2_2
    dsbcop += alpha_2_21_2 + alpha_2_22_2 <= alpha2_2
    dsbcop += alpha_3_11_2 + alpha_3_12_2 <= alpha3_2
    dsbcop += alpha_3_21_2 + alpha_3_22_2 <= alpha3_2
    dsbcop += alpha_2_11_3 + alpha_2_12_3 <= alpha2_3
    dsbcop += alpha_2_21_3 + alpha_2_22_3 <= alpha2_3
    dsbcop += alpha_3_11_3 + alpha_3_12_3 <= alpha3_3
    dsbcop += alpha_3_21_3 + alpha_3_22_3 <= alpha3_3
    # C11
    dsbcop += A_212_21 + 2 * A_22_23 + A_312_21 <= 2 * (A_12_12 + A_12_13)
    dsbcop += A_212_31 + 2 * A_22_32 + A_312_31 <= 2 * (A_12_12 + A_12_13) 
    # C12
    epsilon = 10 ** (-6)
    dsbcop += A_211_21 + A_11_23 + epsilon <= A_212_21 + A_21_23
    dsbcop += A_211_21 + A_12_23 + epsilon <= A_212_21 + A_22_23
    dsbcop += A_211_31 + A_11_32 + epsilon <= A_212_31 + A_12_32
    dsbcop += A_211_31 + A_21_32 + epsilon <= A_212_31 + A_22_32
    dsbcop += A_311_21 + A_11_23 + epsilon <= A_312_21 + A_21_23
    dsbcop += A_311_21 + A_12_23 + epsilon <= A_312_21 + A_22_23
    dsbcop += A_311_31 + A_11_32 + epsilon <= A_312_31 + A_12_32
    dsbcop += A_311_31 + A_21_32 + epsilon <= A_312_31 + A_22_32
        
    # Solve the problem
    dsbcop.solve()

    # Give different returns based on feasibility
    if LpStatus[dsbcop.status] == 'Optimal':
        A_2_21 = np.array([value(A_211_21), value(A_212_21), value(A_221_21), value(A_222_21)])
        A_3_21 = np.array([value(A_311_21), value(A_312_21), value(A_321_21), value(A_322_21)])
        A_2_31 = np.array([value(A_211_31), value(A_212_31), value(A_221_31), value(A_222_31)])
        A_3_31 = np.array([value(A_311_31), value(A_312_31), value(A_321_31), value(A_322_31)])
        return value(dsbcop.objective), (('e_1', A_2_21, A_2_31), ('e_1', A_3_21, A_3_31))
    else:
        return LpStatus[dsbcop.status]

In [16]:
for _ in range(500): 
    # random matrices for exhaustion
    A_12, A_13, A_21, A_23, A_31, A_32 = [np.random.rand(2, 2) for _ in range(6)]
    DSOP = dsop_test(A_12, A_13, A_21, A_23, A_31, A_32)
    DSBCOP = dsbcop_test(A_12, A_13, A_21, A_23, A_31, A_32)
    if isinstance(DSOP, tuple):
        if isinstance(DSBCOP, tuple):
            if DSOP[0] < DSBCOP[0]:
                print('Conjecture 2 supported with example:', '\n', 
                      'A_12=', A_12, 'A_13=', A_13, 'A_21=', A_21, 'A_23=', A_23, 'A_31=', A_31, 'A_32=', A_32, '\n',
                      'with results: ', '\n',
                     'dsop value:', DSOP[0], 'dsbcop value:', DSBCOP[0], '\n',
                      'with strategies:', '\n'
                      'dsop: ', DSOP[1], 'dsbcop:', DSBCOP[1]
                     )
            elif DSOP[0] > DSBCOP[0]:
                print('Conjecture 2 disproved with example:', '\n', 
                      'A_12=', A_12, 'A_13=', A_13, 'A_21=', A_21, 'A_23=', A_23, 'A_31=', A_31, 'A_32=', A_32, '\n',
                      'with results: ', '\n',
                     'dsop value:', DSOP[0], 'dsbcop value:', DSBCOP[0], '\n',
                      'with strategies:', '\n'
                      'dsop: ', DSOP[1], 'dsbcop:', DSBCOP[1]
                     )
                break
            elif DSOP[0] == DSBCOP[0] == 0.0:
                print('no change is the best')
            else:
                print('two approaches equally optimal with example:', '\n', 
                      'A_12=', A_12, 'A_13=', A_13, 'A_21=', A_21, 'A_23=', A_23, 'A_31=', A_31, 'A_32=', A_32, '\n',
                      'with results: ', '\n',
                     'dsop value:', DSOP[0], 'dsbcop value:', DSBCOP[0], '\n',
                      'with strategies:', '\n'
                      'dsop: ', DSOP[1], 'dsbcop:', DSBCOP[1]
                     )
        else: 
            print('dsbcop does not exist for', 'A_12=', A_12, 'A_13=', A_13, 'A_21=', A_21, 'A_23=', A_23, 'A_31=', A_31, 'A_32=', 'A_32')

no change is the best
no change is the best
no change is the best
no change is the best
no change is the best
no change is the best
no change is the best
no change is the best
no change is the best
no change is the best
no change is the best
no change is the best
